In [17]:
CSV_IN = 'datos/Base datos_INTA_ascii_limpio.csv'

In [18]:
print 'Cargando datos de "%s"' % CSV_IN
df = pd.read_csv(CSV_IN)
print 'Cargado dataframe de tamaño %d x %d' % df.shape

Cargando datos de "datos/Base datos_INTA_ascii_limpio.csv"
Cargado dataframe de tamaño 423 x 35


In [19]:
# Estos son funciones para hacer cuentas claras y minimalistas

def truncate_str(s, n=15):
    s = str(s)
    if len(s) > n:
        return s[:(n-3)] + '...'
    else:
        return s

def smart_value_counts(s, top=10, normalize=True, percent=False):
    vc = s.value_counts(ascending=False, normalize=normalize)
    vc.index = [truncate_str(i) for i in vc.index]
    out = vc[:top]
    if vc.sum() > out.sum():
        other = pd.Series([vc.sum() - out.sum()], index=['[OTROS] (%d tipos)' % (len(vc) - len(out))])
        out = out.append(other)
    na = 1.0 - vc.sum()
    out = out.append(pd.Series([na], index=['[NA (%.0f%%)]' % (na * 100)]))
    if percent:
        return out.apply(lambda p: '%.2f%%' % (p*100))
    return out

smart_value_counts(df['campana'], top=5, normalize=True, percent=True)

2011-01-01           17.49%
2012-01-01           14.89%
2014-01-01           13.00%
2009-01-01           12.77%
2008-01-01           12.53%
[OTROS] (3 tipos)    29.31%
[NA (0%)]             0.00%
dtype: object

In [20]:
# df.describe()

In [21]:
smart_value_counts(df['campana'])

2011-01-01    0.174941
2012-01-01    0.148936
2014-01-01    0.130024
2009-01-01    0.127660
2008-01-01    0.125296
2010-01-01    0.101655
2006-01-01    0.101655
2007-01-01    0.089835
[NA (0%)]     0.000000
dtype: float64

In [22]:
def figura_hist(s, fig, ax):
    s = s.astype(float)
    s.hist(ax=ax, color=COLOR_HIST, weights=np.zeros(s.count()) + 1. / len(s),
           alpha=BAR_ALPHA)
    ax.set_yticklabels(['%.0f%%' % (t*100) for t in ax.get_yticks()])
    ax.set_title(c)
    na = 1.0 - float(s.count()) / len(s)
    ax.plot([min(ax.get_xticks()), max(ax.get_xticks())], [na, na],
            color=COLOR_NA, label='NA (%.0f%%)' % (na * 100), alpha=0.8,
            linestyle='--')
    ax.legend(loc='upper right')
    if RUGPLOT:
        sns.rugplot(s, color=COLOR_HIST, alpha=1.0, ax=ax)
    fig.savefig('figuras/explorar_hist_%s.png' % figname)

def figura_bar(s, fig, ax):
    s = smart_value_counts(s)
    s[::-1].plot(ax=ax, kind='barh', color=[COLOR_NA] +
                 [COLOR_BAR]*(len(s) - 1), alpha=BAR_ALPHA)
    ax.set_xticklabels(['%.0f%%' % (t*100) for t in ax.get_xticks()])
    fig.savefig('figuras/explorar_bar_%s.png' % figname)

In [23]:
# Creamos una figura por cada columno, sumarizandolo visualmente

# Configuraction visual
COLOR_HIST = sns.color_palette()[0]
COLOR_BAR = sns.color_palette()[1]
COLOR_NA = (.8, .2, .2)
BAR_ALPHA = 0.8
RUGPLOT = True

saltados, numericos = 0, 0
for i, c in enumerate(sorted(df.columns)):
    if df[c].count() == 0:
        print '**SALTANDO "%s"** (todos NA)' % c
        saltados += 1
        continue
    print '[%d/%d] %s' % (i+1, len(df.columns), c),
    
    fig, ax = plt.subplots(1)
    fig.subplots_adjust(left=0.2)
    ax.set_title(c)
    s = df[c]
    figname = ''.join([ch for ch in c.lower().replace(' ', '_')
                       if ch.isalnum() or ch in ('_-')])
    try:
        figura_hist(s, fig, ax)
        numericos += 1
        print '(float)'
    except ValueError:
        figura_bar(s, fig, ax)
        print '(categorico)'
    plt.close()

print 'Columnos: [%d numericos, %d categoricos, %d saltados (vacios), %d total]' % (
    numericos, len(df.columns) - numericos - saltados, saltados, len(df.columns))

[1/35] analisis_suelo_k (float)
[2/35] analisis_suelo_n (float)
[3/35] analisis_suelo_p (float)
[4/35] campana (categorico)
[5/35] cultivo_antecesor (categorico)
[6/35] densidad_siembra_kg_ha (float)
[7/35] factor_rend_densidad_pl_m2 (categorico)
[8/35] factor_rend_pctfactor (float)
[9/35] factor_rend_pcthcosecha (float)
[10/35] fenologia_cosecha_fecha (categorico)
[11/35] fenologia_dpf_fecha (categorico)
[12/35] fenologia_emerg_cosecha_dias (float)
[13/35] fenologia_emerg_dpf_dias (float)
[14/35] fenologia_emerg_flor_50pct_dias (float)
[15/35] fenologia_emergencia_70pct_fecha (categorico)
[16/35] fenologia_flor_50pct_fecha (categorico)
[17/35] fenologia_siembra_fecha (categorico)
[18/35] fert_cober_clk_cant_kg_ha (float)
[19/35] fert_cober_dpf_cant_n_kg_ha (float)
[20/35] fert_cober_dpf_fecha (categorico)
[21/35] fert_cober_preriego_cant_n_kg_ha (float)
[22/35] fert_cober_preriego_fecha (categorico)
[23/35] fert_cober_preriego_formulado (categorico)
[24/35] id (categorico)
[25/35] ren

In [24]:
sumario = df.describe()
sumario

,densidad_siembra_kg_ha,analisis_suelo_n,analisis_suelo_p,analisis_suelo_k,fert_cober_preriego_cant_n_kg_ha,fert_cober_dpf_cant_n_kg_ha,fert_cober_clk_cant_kg_ha,fenologia_emerg_dpf_dias,riego_velocidad_dias,riego_emerg_lote_completo_dias,fenologia_emerg_flor_50pct_dias,fenologia_emerg_cosecha_dias,factor_rend_pcthcosecha,rendimiento_seco_kg_ha,factor_rend_pctfactor,rinde_con_factor_kg_ha,superficie_has
count,423.000000,372.000000,372.000000,361.000000,311.00000,250.000000,114.000000,249.00000,385.000000,385.000000,423.000000,385.000000,382.000000,423.000000,327.000000,330.000000,423.000000
mean,103.966903,10.828387,23.075827,37.780709,47.16701,32.572800,54.433368,52.48996,5.529870,26.677922,90.846336,131.137662,20.229843,7006.091686,0.970690,6960.750929,38.653097
std,14.632750,6.210559,13.491036,19.829912,10.68954,10.774992,26.251850,6.01216,2.997354,7.775506,8.282485,10.872809,3.437030,1451.891081,0.090794,1651.947275,17.409731
min,45.000000,3.700000,8.720000,2.490000,0.00000,23.000000,24.900000,40.00000,0.000000,2.000000,72.000000,107.000000,12.000000,2478.160920,0.523500,2427.358621,0.900000
25%,98.000000,7.000000,13.080000,29.880000,46.00000,27.600000,39.840000,45.00000,4.000000,22.000000,84.500000,124.000000,18.000000,6010.236697,0.933000,5741.582064,27.700000
50%,100.000000,7.500000,19.349680,35.400000,46.00000,27.600000,39.840000,53.00000,5.000000,26.000000,91.000000,131.000000,20.400000,7158.106371,0.997500,7228.050798,37.000000
75%,110.000000,20.000000,23.544000,43.160000,46.00000,36.800000,80.000000,58.00000,7.000000,32.000000,97.000000,138.000000,22.700000,8067.185152,1.030500,8203.673237,47.400000
max,162.000000,25.200000,63.000000,86.320000,101.20000,69.000000,100.000000,64.00000,34.000000,54.000000,112.000000,163.000000,29.500000,10905.340622,1.101000,10981.678007,123.000000
